In [22]:
from openbb import obb
import pandas as pd

obb.user.preferences.output_type = 'dataframe'
asset = obb.equity.price.historical('AAPL', provider='yfinance')
benchmark = obb.equity.price.historical('SPY', provider='yfinance')

columns = ['open', 'high', 'low', 'close', 'volume', 'dividends']
asset.columns = columns # rename columns
benchmark.columns = columns 

# add columns with values from aggregate
asset['price_diff'] = asset.close.diff()
benchmark['price_diff'] = benchmark.close.diff()

# add a column with a boolean value
asset['gain'] = asset.price_diff > 0
benchmark['gain'] = benchmark.price_diff > 0

# add a colmn with a symbol string
asset['symbol'] = 'AAPL'
benchmark['symbol'] = 'SPY'

# set a single value based on aggregation
asset_2 = asset.copy()
asset_2['volume'] = asset_2['volume'].astype(float)
asset_2.at[asset_2.index[1], 'volume'] = asset_2.volume.mean()

# concatenate two dataframes together
assets = pd.concat([asset, asset_2]).drop_duplicates()

In [36]:
from openbb import obb
import pandas as prd 

obb.user.preferences.output_type = 'dataframe'
chains = obb.derivatives.options.chains('AAPL', provider='cboe')

expirations = chains.expiration.unique()
calls = chains[(chains.option_type == 'call') & (chains.expiration == expirations[5])]
put = chains[(chains.option_type == 'put') & (chains.expiration == expirations[5])]

call_strike = calls.set_index('strike')
put_strike = put.set_index('strike')

joined = call_strike.join(put_strike, how='left', lsuffix='_call', rsuffix='_put')
prices = joined[['last_trade_price_call', 'last_trade_price_put']]
prices['straddle_price'] = prices.sum(axis=1)
print(prices)

        last_trade_price_call  last_trade_price_put  straddle_price
strike                                                             
100.0                    0.00                  0.03            0.03
110.0                    0.00                  0.04            0.04
120.0                    0.00                  0.06            0.06
130.0                    0.00                  0.00            0.00
140.0                    0.00                  0.10            0.10
145.0                    0.00                  0.10            0.10
150.0                    0.00                  0.10            0.10
155.0                   76.10                  0.12           76.22
160.0                    0.00                  0.18            0.18
165.0                    0.00                  0.16            0.16
170.0                    0.00                  0.18            0.18
175.0                    0.00                  0.19            0.19
180.0                   53.91                  0

/tmp/ipykernel_12038/478846630.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['straddle_price'] = prices.sum(axis=1)
